<a href="https://colab.research.google.com/github/sumeetku/test/blob/master/modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ls

drive/  sample_data/


In [6]:
import torch
from fastai.vision import *
from pathlib import Path
from fastai.vision import *
import pandas as pd

In [7]:
from fastai.vision import *
from pathlib import Path
import pandas as pd

In [8]:
from IPython.core.display import display, HTML

In [9]:
path=Path('/content/drive/My Drive/Colab Notebooks/bbc_train/')

In [10]:
ls

drive/  sample_data/


In [11]:
labels=pd.read_csv(path/'classification_labels.csv')

In [12]:
labels.head()

,filename,label,label_index
0,P2050936.jpg,ball,2
1,P2050922.jpg,ball,2
2,P2060163.jpg,brick,1
3,P2060003.jpg,ball,2
4,P2060017.jpg,ball,2


In [146]:
images, lbl_bbox = get_annotations(path/'bboxes.json')
img2bbox = dict(zip(images, lbl_bbox))
get_y_func = lambda o:img2bbox[o.name]
src = ObjectItemList.from_folder(path/'images')
def get_data(bs, size):
    src = ObjectItemList.from_folder(path/'images')
    src = src.split_by_rand_pct(0.2)
    src = src.label_from_func(get_y_func)
    src = src.transform(get_transforms(max_rotate=0), size=size, tfm_y=True)
    return src.databunch(path=path, bs=bs, collate_fn=bb_pad_collate)

data = get_data(bs=64,size=256)

In [147]:
class MyLoss(nn.Module):
    def forward(self, yhat, bbox_tgts, class_tgts):
        det_loss=nn.L1Loss()(yhat[:,:4].unsqueeze_(dim=1), bbox_tgts)
        cls_loss=nn.CrossEntropyLoss()(yhat[:,4:], class_tgts.view(-1))
        #print(det_loss, cls_loss)
        
        return det_loss + 1.0*cls_loss
def my_accuracy(yhat, bbox_tgts, class_tgts):
    return accuracy(yhat[:,4:], class_tgts.view(-1))

def my_l1(yhat, bbox_tgts, class_tgts):
    return nn.L1Loss()(yhat[:,:4].unsqueeze_(dim=1), bbox_tgts)



In [148]:
custom_head = nn.Sequential(
    AdaptiveConcatPool2d((4,4)),
    Flatten(),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(16384,256), #2*512*4*4
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),
    nn.Linear(256,8))

In [149]:
learn=cnn_learner(data, models.resnet34, custom_head=custom_head, loss_func=MyLoss(), metrics=[my_accuracy, my_l1])

In [160]:

lr=1e-2
learn.fit_one_cycle(20, slice(lr/10, lr))

epoch,train_loss,valid_loss,my_accuracy,my_l1,time
0,0.464330,0.321213,0.989362,0.179978,00:14
1,0.450926,0.322436,0.984043,0.182686,00:14
2,0.413591,0.313928,0.984043,0.165134,00:14
3,0.370049,0.287921,0.989362,0.157747,00:14
4,0.335508,0.268868,0.994681,0.148755,00:14
5,0.318850,0.268843,0.989362,0.139999,00:14
6,0.300885,0.247073,0.989362,0.125979,00:14
7,0.283735,0.232442,0.994681,0.120483,00:15
8,0.264628,0.243878,0.989362,0.119848,00:14
9,0.254126,0.243310,0.994681,0.124032,00:15


In [169]:
lr=5e-4
learn.fit_one_cycle(30, slice(lr/10, lr))

epoch,train_loss,valid_loss,my_accuracy,my_l1,time
0,0.116977,0.169391,0.994681,0.067301,00:15
1,0.117733,0.168375,0.994681,0.067173,00:15
2,0.112997,0.167989,0.994681,0.067023,00:15
3,0.113358,0.168793,0.994681,0.066714,00:15
4,0.112527,0.168935,0.994681,0.067861,00:15
5,0.111730,0.168868,0.994681,0.068650,00:15
6,0.111255,0.164985,0.994681,0.067420,00:14
7,0.109375,0.164444,0.994681,0.066953,00:14
8,0.109491,0.164888,0.994681,0.066873,00:14
9,0.110173,0.165236,0.994681,0.068265,00:14


In [175]:
learn.export()

In [100]:
learn.path

PosixPath('/content/drive/My Drive/Colab Notebooks/bbc_train')

In [171]:
!cp /content/drive/MyDrive/Colab\ Notebooks/bbc_train/export.pkl /content/

In [85]:
learn.path

PosixPath('/content/drive/My Drive/Colab Notebooks/bbc_train')

In [173]:
!python3 eval.py

1. Loading Data...
ImageDataBunch;

Train: LabelList (942 items)
x: SegmentationItemList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: SegmentationLabelList
ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256)
Path: /content/drive/MyDrive/Colab Notebooks/bbc_train/images;

Valid: LabelList (0 items)
x: SegmentationItemList

y: SegmentationLabelList

Path: /content/drive/MyDrive/Colab Notebooks/bbc_train/images;

Test: None
2. Instantiating Model...
3. Evaluating on Test Images...

Done!
Execution time (seconds) =  8.634
Score =  7 
Combined accuracy =  0.9684456969068075 
Classification accuracy =  1.0 
bbox score =  0.9645926906960085 
Segmentation Accuracy =  0.9407444000244141
